In [37]:
import pandas as pd
import re

In [2]:
base = pd.read_csv('../API/MoviesPopular.csv')
print(base.columns)

Index(['Unnamed: 0', 'adult', 'backdrop_path', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count'],
      dtype='object')


In [3]:
#Chargement des bases scrappées sur wikipédia pour nettoyage et fusion avec la base de données récupérée sur TMDB

#Academy award for best cinematography 
ac_bc = pd.read_csv('../wiki_scrap/academy_bc.csv')
# academy award for best film 
ac_bf = pd.read_csv('../wiki_scrap/academy_bf.csv')
#Academy award for best director 
ac_bd = pd.read_csv('../wiki_scrap/academy_bd.csv')
#Cannes Grand Prix
can_gp = pd.read_csv('../wiki_scrap/cannes_gp.csv')
#Cannes Jury prize
can_jr = pd.read_csv('../wiki_scrap/cannes_jury.csv')
#Cannes Palme d'or
can_po = pd.read_csv('../wiki_scrap/palme_or.csv')
#César Best director 
ces_bd = pd.read_csv('../wiki_scrap/cesar_bd.csv')
#Cesar Best Film 
ces_bf = pd.read_csv('../wiki_scrap/cesar_bf.csv')
#Venice Lion d'or
ven_lor = pd.read_csv('../wiki_scrap/lion_or.csv')
#Venice Lion d'argent 
ven_larg = pd.read_csv('../wiki_scrap/lion_argent.csv')
#Venice Grand Jury Prix
ven_gj = pd.read_csv('../wiki_scrap/venice_gj.csv')


In [4]:
#Pour vérifier si le chargement est bien réussi 

print(ac_bc.columns)
print(ac_bd.columns) 
print(ac_bf.columns) 

print(ces_bd.columns) 
print(ces_bf.columns) 

print(can_gp.columns)
print(can_jr.columns) 
print(can_po.columns)

print(ven_lor.columns) 
print(ven_larg.columns) 
print(ven_gj.columns)

Index(['Category', 'Name', 'Superlative', 'Year', 'Notes', 'Film', 'Nominees',
       'Awards', 'Nominations', 'Recipient'],
      dtype='object')
Index(['0', 'Year', 'Director(s)', 'Film', 'Ref.', 'Wins', 'Director',
       'Nominations', 'Record', 'Age'],
      dtype='object')
Index(['Year of Film Release', 'Film', 'Film Studio',
       'Film Studio/Producer(s)', 'Producer(s)', 'Production Company',
       'Nominations', 'Wins'],
      dtype='object')
Index(['Superlative', 'Men', 'Men.1', 'Women', 'Women.1', 'Year',
       'Winner and nominees', 'English title', 'Original title',
       'Winners and nominees', 'Wins', 'Director', 'Nominations'],
      dtype='object')
Index(['Year', 'English title', 'Original title', 'Director'], dtype='object')
Index(['('Year', '1960s', 'Awarded as "Grand Prix Spécial du Jury"')',
       '('English Title', '1960s', 'Awarded as "Grand Prix Spécial du Jury"')',
       '('Original Title', '1960s', 'Awarded as "Grand Prix Spécial du Jury"')',
       '('D

In [20]:
#Nettoyage et mise en conformité de la base de données Academy 

#Best-cinematography 

#Supprimer les 3 premières et 3 dernières colonnes et 13 premières lignes qui référent à un tableau qui ne nous intéresse pas dans notre code
ac_bc_cl = ac_bc.drop(['Category', 'Name', 'Superlative', 'Notes', 'Awards', 'Nominations', 'Recipient'], axis=1)

ac_bc_cl = ac_bc_cl.iloc[13:, :]


#suppression lignes avec seulement année 

ac_bc_cl['Year'] = ac_bc_cl['Year'].astype(str)

#conversion en type date de la colonne 'Year' 
ac_bc_cl['Year'] = ac_bc_cl['Year'].str.extract('(\d{4})', expand=False)
ac_bc_cl['Year'] = pd.to_datetime(ac_bc_cl['Year'], format='%Y', errors='coerce')
ac_bc_cl = ac_bc_cl.dropna(subset=['Film', 'Nominees'], how='all')

ac_bc_cl



,Year,Film,Nominees
13,1927-01-01,Sunrise: A Song of Two Humans,Charles Rosher Karl Struss
14,1927-01-01,The Devil Dancer,George Barnes
15,1927-01-01,The Magic Flame,George Barnes
16,1927-01-01,Sadie Thompson,George Barnes
18,1928-01-01,White Shadows in the South Seas,Clyde De Vinna
...,...,...,...
782,2022-01-01,All Quiet on the Western Front,James Friend
783,2022-01-01,"Bardo, False Chronicle of a Handful of Truths",Darius Khondji
784,2022-01-01,Elvis,Mandy Walker
785,2022-01-01,Empire of Light,Roger Deakins


In [23]:
ac_bd_cl = ac_bd.drop (['0', 'Ref.', 'Wins', 'Director', 'Nominations', 'Record', 'Age' ], axis=1)
ac_bd_cl = ac_bd_cl.iloc[:473, :]

ac_bd_cl['Year'] = ac_bd_cl['Year'].astype(str)
ac_bd_cl['Year'] = ac_bd_cl['Year'].str.extract('(\d{4})', expand=False)
ac_bd_cl['Year'] = pd.to_datetime(ac_bd_cl['Year'], format='%Y', errors='coerce')
ac_bd_cl.rename(columns={'Director(s)': 'Director'}, inplace=True)
ac_bd_cl = ac_bd_cl.dropna(subset=['Director', 'Film'], how='all')

ac_bd_cl



,Year,Director,Film
1,1927-01-01,Frank Borzage (Dramatic Picture),7th Heaven
2,1927-01-01,Herbert Brenon (Dramatic Picture),Sorrell and Son
3,1927-01-01,King Vidor (Dramatic Picture),The Crowd
4,1927-01-01,Lewis Milestone (Comedy Picture),Two Arabian Knights
5,1927-01-01,Ted Wilde (Comedy Picture),Speedy
...,...,...,...
468,2022-01-01,Daniel Kwan and Daniel Scheinert,Everything Everywhere All at Once
469,2022-01-01,Todd Field,Tár
470,2022-01-01,Martin McDonagh,The Banshees of Inisherin
471,2022-01-01,Ruben Östlund,Triangle of Sadness


In [43]:
ac_bf_cl=ac_bf

def extract_studio_producer(entry):
    if pd.notna(entry):
        match = re.match(r'(.+?)\s*\((.+)\)', str(entry))
        if match:
            return match.group(1).strip(), match.group(2).strip()
        else:
            return entry.strip(), None
    else:
        return entry, None
        
# Apply the function to the 'Film Studio' column
ac_bf_cl[['Studio', 'Producer']] = ac_bf_cl['Film Studio'].apply(extract_studio_producer).apply(pd.Series)

ac_bf_cl['Film Studio'] = ac_bf_cl['Studio']

# Fill missing 'Producer(s)' values using 'Producer' only when 'Producer(s)' is NaN
mask = ac_bf_cl['Producer(s)'].isna()
ac_bf_cl.loc[mask, 'Producer(s)'] = ac_bf_cl['Producer']

# Drop the intermediate 'Studio' and 'Producer' columns
ac_bf_cl = ac_bf_cl.drop(columns=['Studio', 'Producer'])
# Display the DataFrame after processing
print("\nDataFrame after processing:")
ac_bf_cl


DataFrame after processing:


,Year of Film Release,Film,Film Studio,Film Studio/Producer(s),Producer(s),Production Company,Nominations,Wins,StudioName
0,1927/28 (1st),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1927/28 (1st),Wings,Famous Players–Lasky,NaN,"Lucien Hubbard, Jesse L. Lasky, B.P. Schulberg...",NaN,NaN,NaN,NaN
2,1927/28 (1st),7th Heaven,Fox,NaN,"William Fox, producer",NaN,NaN,NaN,NaN
3,1927/28 (1st),The Racket,The Caddo Company,NaN,"Howard Hughes, producer",NaN,NaN,NaN,NaN
4,1928/29 (2nd) [a],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
714,NaN,NaN,NaN,NaN,NaN,Pixar Animation Studios,2.0,0.0,NaN
715,NaN,NaN,NaN,NaN,NaN,Hollywood Pictures,2.0,0.0,NaN
716,NaN,NaN,NaN,NaN,NaN,The Caddo Company,2.0,0.0,NaN
717,NaN,NaN,NaN,NaN,NaN,Walter Wanger (production company),2.0,0.0,NaN
